[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: Jueves 01 de mayo del 2025 - hasta las 17:00hs.**
 
<span style="color: red;">**Fecha de entrega intermedia: Jueves 17 de Abril del 2025 - hasta las 17:00hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 5). 

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido. 

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio. 
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

**El informe no puede superar un máximo de 8 carillas (contando gráficos) o 4 hojas más carátula.** Tamaño de la letra: estandár de latex (10pt). No se corregirán trabajos que no cumplan con esta consigna.

---

In [37]:
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold,RandomizedSearchCV,ParameterGrid,cross_val_predict
from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score,average_precision_score

## Ejercicio 1 

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. 

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

In [38]:
df=pd.read_csv("data.csv")
df.head()
df.shape

(500, 201)

In [39]:
df_c1 = df[df['target'] == 1]
df_c2 = df[df['target'] == 0]

# Mezclar aleatoriamente
df_c1 = df_c1.sample(frac=1, random_state=42).reset_index(drop=True)
df_c2 = df_c2.sample(frac=1, random_state=42).reset_index(drop=True)

# Tamaño total del dataset
n_total = len(df)
n_eval = int(0.2 * n_total)  # 20% para evaluación

# Cantidades por clase para evaluación
n_eval_c1 = int(0.2 * len(df_c1))  # 30
n_eval_c2 = n_eval - n_eval_c1     # 70

# Separar evaluación
df_eval_c1 = df_c1.iloc[:n_eval_c1]
df_eval_c2 = df_c2.iloc[:n_eval_c2]

# Lo que queda es desarrollo
df_dev_c1 = df_c1.iloc[n_eval_c1:]
df_dev_c2 = df_c2.iloc[n_eval_c2:]

# Combinar y mezclar
df_eval = pd.concat([df_eval_c1, df_eval_c2]).sample(frac=1, random_state=42).reset_index(drop=True)
df_dev = pd.concat([df_dev_c1, df_dev_c2]).sample(frac=1, random_state=42).reset_index(drop=True)

X_dev = df_dev.drop(columns=["target"])
y_dev = df_dev["target"]
X_eval=df_eval.drop(columns=["target"])
y_eval=df_eval["target"]

### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

In [40]:
tree_model=DecisionTreeClassifier(max_depth=3,random_state=42)


1. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_, y _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.
   
   Reportar el resultado en una tabla similar a:

      <table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td>0.83125</td>
      <td>0.675</td>
      <td>0.68118773</td>
      <td>0.34724893</td>
      <td>0.82934215</td>
      <td>0.57738095</td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td>0.784375</td>
      <td>0.6</td>
      <td>0.65482599</td>
      <td>0.37581619</td>
      <td>0.83172299</td>
      <td> 0.62165179</td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td>0.790625</td>
      <td>0.7375</td>
      <td>0.66016275</td>
      <td>0.43671292</td>
      <td>0.83026675</td>
      <td>0.6499256</td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td>0.853125</td>
      <td>0.6125</td>
      <td>0.74582953</td>
      <td>0.27594626</td>
      <td>0.83239332</td>
      <td>0.4375</td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td>0.784375</td>
      <td>0.7</td>
      <td>0.65221835</td>
      <td>0.40623895</td>
      <td>0.81794085</td>
      <td>0.64763636</td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td>0.8087500000000001</td>
      <td>0.665</td>
      <td>0.6788448710972432</td>
      <td>0.3683926507628855</td>
      <td>0.82833321148096</td>
      <td>0.5868189393939394</td>
      </tr>
      <td align="center">Global</td>
      <td>(NO) </td>
      <td>0.665</td>
      <td>(NO) </td>
      <td>0.3652341593248588</td>
      <td>(NO) </td>
      <td>0.6109185698628515</td>
      </tr>
      </tbody>
      </table>    
  

In [41]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluación con múltiples métricas
scoring = {
    'accuracy': 'accuracy',
    'auprc': 'average_precision',
    'aucroc': 'roc_auc'
}
scores = cross_validate(
    tree_model,
    X_dev,
    y_dev,
    cv=skf,
    scoring=scoring,
    return_train_score=True
)
train_accuracy=scores['train_accuracy']
test_accuracy=scores["test_accuracy"]
print(f"El train_accuracy es {train_accuracy} con promedio {train_accuracy.mean()} y el test_accuracy es {test_accuracy} con promedio {test_accuracy.mean()}") 
train_auprc=scores['train_auprc']
test_auprc=scores['test_auprc']
print(f"El train_auprc es {train_auprc} con promedio {train_auprc.mean()} y el test_auprc es {test_auprc} con promedio {test_auprc.mean()}") 
train_aucroc=scores['train_aucroc']
test_aucroc=scores["test_aucroc"]
print(f"El train_aucroc es {train_aucroc} con promedio {train_aucroc.mean()} y el test_accuracy es {test_aucroc} con promedio {test_aucroc.mean()}") 

El train_accuracy es [0.83125  0.784375 0.790625 0.853125 0.784375] con promedio 0.8087500000000001 y el test_accuracy es [0.675  0.6    0.7375 0.6125 0.7   ] con promedio 0.665
El train_auprc es [0.68118773 0.65482599 0.66016275 0.74582953 0.65221835] con promedio 0.6788448710972432 y el test_auprc es [0.34724893 0.37581619 0.43671292 0.27594626 0.40623895] con promedio 0.3683926507628855
El train_aucroc es [0.82934215 0.83172299 0.83026675 0.83239332 0.81794085] con promedio 0.82833321148096 y el test_accuracy es [0.57738095 0.62165179 0.6499256  0.4375     0.64763636] con promedio 0.5868189393939394


In [42]:
#Ahora calculamos los scores globales sobre los conjuntos de validacion
y_pred = cross_val_predict(tree_model, X_dev, y_dev, cv=skf)
print(f"El accuracy global para los datos de validacion es {accuracy_score(y_dev,y_pred)}")
y_proba = cross_val_predict(tree_model, X_dev, y_dev, cv=skf, method='predict_proba')
y_scores=y_proba[:,1]
print(f"El AUPRC global para los datos de validacion es {average_precision_score(y_dev,y_scores)}")
print(f"El AUROC global para los datos de validacion es {roc_auc_score(y_dev,y_scores)}")

El accuracy global para los datos de validacion es 0.665
El AUPRC global para los datos de validacion es 0.3652341593248588
El AUROC global para los datos de validacion es 0.6109185698628515


*INCISO 3*
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.
1. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td>0.8087500000000001</td>
   <td>0.6575</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td>0.9106249999999999</td>
   <td>0.64</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td>1.0</td>
   <td>0.64</td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td>0.796875</td>
   <td>0.685</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td>0.89875</td>
   <td>0.6525</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td>1.0</td>
   <td>0.66</td>
   </tr>
   </tbody></table>

1. ¿Qué conclusiones se pueden sacar de estas tablas? 

In [43]:
param_grid={"max_depth":[3,5,None],"criterion":["gini","entropy"]}
grid=list(ParameterGrid(param_grid))
for param in grid:
    tree_model=tree_model=DecisionTreeClassifier(**param)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores=cross_validate(
    tree_model,
    X_dev,
    y_dev,
    cv=skf,
    scoring="accuracy",
    return_train_score=True
    )
    print(f"Con los parametros {param} obtuvimos un train_accuracy={scores['train_score'].mean()} y un validation_acurracy={scores['test_score'].mean()}")
    

Con los parametros {'criterion': 'gini', 'max_depth': 3} obtuvimos un train_accuracy=0.8087500000000001 y un validation_acurracy=0.6625
Con los parametros {'criterion': 'gini', 'max_depth': 5} obtuvimos un train_accuracy=0.913125 y un validation_acurracy=0.635
Con los parametros {'criterion': 'gini', 'max_depth': None} obtuvimos un train_accuracy=1.0 y un validation_acurracy=0.635
Con los parametros {'criterion': 'entropy', 'max_depth': 3} obtuvimos un train_accuracy=0.796875 y un validation_acurracy=0.685
Con los parametros {'criterion': 'entropy', 'max_depth': 5} obtuvimos un train_accuracy=0.89875 y un validation_acurracy=0.665
Con los parametros {'criterion': 'entropy', 'max_depth': None} obtuvimos un train_accuracy=1.0 y un validation_acurracy=0.6425000000000001


Observamos que los modelos tienden a overfittear y que esto es mas evidente a medida que el arbol aumenta su profundidad en el caso de $depth=\infty$ tenemos un accuracy 1 en el train y un accuracy mucho mas bajo en el test. Tambien pareceria no haber mucha diferencia entre los criterios de eleccion.

## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar: 
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo). 

2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default. 

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

3. ¿Cuál fue el mejor modelo y con qué configuración? Explicar por qué creería que dio mejor (recordando qué hace cada algoritmo y con qué tipo de datos están trabajando).

## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar: 
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo). 

In [44]:
from scipy.stats import randint

Primero lo hacemos para arboles de decision, aca seguimos maso menos con la misma eleccion de hiperparametros de los ejercicios anteriores, probamos todas los criterios de seleccion para features y permitimos como maximo un depth de 10 para no overfittear.

In [45]:
#Primero lo hacemos para arboles de decision
param_arboles={"criterion":["gini","entropy"],"max_depth":randint(1,10),
               'splitter': ['best', 'random'],"min_samples_leaf":randint(1,10)}
n_iter=30
k=5
tree_model=DecisionTreeClassifier(random_state=42)
random_search=RandomizedSearchCV(tree_model,param_distributions=param_arboles,n_iter=n_iter,cv=k,scoring="roc_auc",
                                 random_state=42)
random_search.fit(X_dev,y_dev)
mejor_modelo=random_search.best_estimator_
print(f"La mejor configuracion de hiperparametros es: {random_search.best_params_}")
print(f"El score promedio del mejor modelo es: {random_search.best_score_}")


La mejor configuracion de hiperparametros es: {'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 8, 'splitter': 'best'}
El score promedio del mejor modelo es: 0.6371971861471861


Ahora para K-NN


In [46]:
from sklearn.neighbors import KNeighborsClassifier
param_knn={'n_neighbors': randint(3, 20),
    'weights': ['uniform', 'distance'],
    'p': [1, 2] }
knn=KNeighborsClassifier()
random_search=RandomizedSearchCV(knn,param_distributions=param_knn,n_iter=n_iter,cv=k,scoring="roc_auc")
random_search.fit(X_dev,y_dev)
print(f"Los mejores hiperparametros son : {random_search.best_params_}")
print(f"El score promedio del mejor modelo es :{random_search.best_score_}")

Los mejores hiperparametros son : {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
El score promedio del mejor modelo es :0.8574021645021646


Vamos con SVM, primero realizamos una estandarizacion de los datos debido a que en SVM se miden distancias (expandirse aca) luego probamos basicamente lo mas relevante para el modelo: el valor de C, aca lo que hicimos fue permitir que una proporcion razonable de los datos puedan estar en el lado incorrecto del margen, dado que tenemos del orden de 400 datos nos parecio prudente tomar como maximo C=50. Por otro lado, probamos kernels gaussianos, lineales y polinomicos, aca no hay mucho misterio; el mejor parametro esta asociado a la separacion de los datos, por ultimo la cantidad de iteraciones fue elegido para que el metodo converga.

In [47]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Definimos el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])


param_svm = [
    {
        "svc__kernel": ["linear"],
        "svc__C": randint(10, 50),
        "svc__max_iter": randint(10000, 20000)
    },
    {
        "svc__kernel": ["poly"],
        "svc__C": randint(10, 50),
        "svc__degree": randint(1, 5),
        "svc__max_iter": randint(10000, 20000)
    },
    {
        "svc__kernel": ["rbf"],
        "svc__C": randint(10, 50),
        "svc__max_iter": randint(10000, 20000)
    }
]

# Grid search
grid_svm = RandomizedSearchCV(pipeline, param_distributions=param_svm,
                              n_iter=n_iter, cv=k, scoring="roc_auc")

# Entrenamiento
grid_svm.fit(X_dev, y_dev)

# Resultados
print(grid_svm.best_params_)
print(grid_svm.best_score_)


{'svc__C': 25, 'svc__degree': 2, 'svc__kernel': 'poly', 'svc__max_iter': 11880}
0.7936307359307359


2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default. 

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

Para LDA seria relevante ver variantes del hiperparametro "solver" que por defecto utiliza una descomposicion SVD para no calcular la matriz de covarianza, otra opcion es "lsqr" o "eigen" que por lo que vimos es util cuando tenemos pocos datos (nuestro caso).
El hiperparametro "shrinkage" podria ser util para evitar el overfitting, el default es None.

In [48]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

gaussian_nb=GaussianNB()
lda=LinearDiscriminantAnalysis()
modelos=[lda,gaussian_nb]
skf = StratifiedKFold(
            n_splits=5,
            shuffle=True,
            random_state=42)
for model in modelos:
    score = cross_validate(
                model,   
                X_dev,
                y_dev,
                cv=skf,
                scoring='roc_auc',
                return_train_score=True
                )
    print(f"El score promedio del train del modelo {model} es {score['train_score'].mean()} y el score promedio del test es {score['test_score'].mean()}")

El score promedio del train del modelo LinearDiscriminantAnalysis() es 0.9956506717238927 y el score promedio del test es 0.724398593073593
El score promedio del train del modelo GaussianNB() es 0.9698246007748195 y el score promedio del test es 0.7474482683982684


## Ejercicio 4
### Diagnóstico Sesgo-Varianza.

<span style="color: red;">(no realizar hasta la clase _Sesgo y Varianza_)</span>

En este punto, se pide inspeccionar **tres** de sus mejores modelos encontrados hasta ahora de cada familia de modelos: la mejor configuración para el árbol de decisión y la mejor configuración para SVM. Para ello:

1. Graficar curvas de complejidad para cada modelo, variando la profundidad en el caso de árboles, y el hiperparámetro C en el caso de SVM. Diagnosticar cómo afectan al sesgo y a la varianza esos dos hiperparámetros.
2. Graficar curvas de aprendizaje para cada modelo pero ahora incluya LDA. En base a estas curvas, sacar conclusiones sobre si los algoritmos parecen haber alcanzado su límite, o bien si aumentar la cantidad de datos debería ayudar.
3. Construir un modelo **RandomForest** con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos.


**Atención**: Tener en cuenta que debemos seguir utilizando AUC ROC como métrica para estas curvas.


## Ejercicio 5: 
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out). 

- Para ello, deberán predecir las **probabilidades** de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos. 

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`. 

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


## Ejercicio 6: 
### Conclusiones

Escribir como mínimo en un párrafo, una conclusión del trabajo realizado, incluyendo problemas encontrados y 
aspectos no incluidos en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio. 
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All). 
    - Las predicciones del *held out* del punto 5 en formato csv.
- Habŕa una entrega intermedia obligatoria que deberán hacer antes del 17 de abril de 2025 a las 17:00hs. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios. 
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de 5 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.